In [3]:
import csv
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
# CARICO I DATI DEI 740 UTENTI FILTRATI IN AUTOMATICO TRA I FRIEND DI PRIMO LIVELLO 
    
filename = "data/friends_1st_level_filtered.csv"   
df = pd.read_csv(filename).drop(["Unnamed: 0","energy","label"], axis=1)

In [6]:
# df.head(5)

In [7]:
# CARICO GLI SCREEN NAME CHE HO SELEZIONATO MANUALMENTE TRA I 740 SELEZIONATI IN AUTOMATICO
filename = "data/friends_1st_level_filtered classification - ScreenNamesOnly.csv"   
sel_df = pd.read_csv(filename,header=None)
sel_df.columns = ['selected_users']

In [8]:
# sel_df

In [9]:
final_df = sel_df.merge(df, left_on="selected_users", right_on='screen_name', how='right')

In [10]:
# final_df[final_df['selected_users'].isnull()]

In [11]:
import numpy
def create_classification(row):
    try:
        numpy.isnan(row['selected_users'])
        return 0
    except:
        return 1

In [12]:
final_df['classification'] = final_df.apply (lambda row: create_classification(row), axis=1)

In [13]:
friends_1st_level_manually_classified_df = final_df.drop(['selected_users'],axis=1)

In [14]:
friends_1st_level_manually_classified_df = friends_1st_level_manually_classified_df.sample(frac=1).reset_index(drop=True)
# friends_1st_level_manually_classified_df

### Verifica delle lingue presenti nel dataset

In [15]:
from langdetect import detect_langs

def add_detected_language_column(row):
    """
    Return detected language only if related probability is greater than 90%
    """
    res = detect_langs(row['description'])
    if res[0].prob>0.9:
        return res[0].lang

In [16]:
friends_1st_level_manually_classified_df['detected_lang'] = friends_1st_level_manually_classified_df.apply (lambda row: add_detected_language_column(row), axis=1)

In [17]:
# friends_1st_level_manually_classified_df[['description','detected_lang']]

In [18]:
# friends_1st_level_manually_classified_df[['description','detected_lang','classification']].groupby(['detected_lang','classification']).count()

In [19]:
friends_1st_level_manually_classified_IT_df = friends_1st_level_manually_classified_df[friends_1st_level_manually_classified_df['detected_lang']=='it']

In [20]:
friends_1st_level_manually_classified_EN_df = friends_1st_level_manually_classified_df[friends_1st_level_manually_classified_df['detected_lang']=='en']

In [21]:
friends_1st_level_manually_classified_IT_df[['description','detected_lang','classification']].groupby(['detected_lang','classification']).count()

description
detected_lang classification             
it            0                       131
              1                        67

In [22]:
friends_1st_level_manually_classified_EN_df[['description','detected_lang','classification']].groupby(['detected_lang','classification']).count()

description
detected_lang classification             
en            0                       222
              1                       259

### Divisione del dataset in dati di training e dati di test

In [23]:
# TUTTO IL DATASET
# dataset_0 = friends_1st_level_manually_classified_df[friends_1st_level_manually_classified_df['classification']==0]
# dataset_1 = friends_1st_level_manually_classified_df[friends_1st_level_manually_classified_df['classification']==1]

# ITALIANO!!!
# dataset_0 = friends_1st_level_manually_classified_IT_df[friends_1st_level_manually_classified_IT_df['classification']==0]
# dataset_1 = friends_1st_level_manually_classified_IT_df[friends_1st_level_manually_classified_IT_df['classification']==1]

# INGLESE!!!
dataset_0 = friends_1st_level_manually_classified_EN_df[friends_1st_level_manually_classified_EN_df['classification']==0]
dataset_1 = friends_1st_level_manually_classified_EN_df[friends_1st_level_manually_classified_EN_df['classification']==1]

In [25]:
train_dataset_0 = dataset_0[:int(dataset_0.shape[0]*0.75)] # Uso 3/4 del dataset per il training
train_dataset_1 = dataset_1[:int(dataset_1.shape[0]*0.75)] # Uso 3/4 del dataset per il training

test_dataset_0 = dataset_0[int(dataset_0.shape[0]*0.75):] # Uso 1/4 del dataset per il test
test_dataset_1 = dataset_1[int(dataset_1.shape[0]*0.75):] # Uso 1/4 del dataset per il test

print(train_dataset_0.shape[0])
print(train_dataset_1.shape[0])
print(test_dataset_0.shape[0])
print(test_dataset_1.shape[0])

166
194
56
65


In [26]:
train_dataset = pd.concat([train_dataset_0,train_dataset_1])
test_dataset = pd.concat([test_dataset_0,test_dataset_1])

In [27]:
test_dataset.shape

(121, 11)

In [24]:
train_dataset.to_csv("train_dataset_EN.csv")

In [28]:
test_dataset.to_csv("test_dataset_EN.csv")

In [23]:
target = test_dataset['classification']
target = list(target.values)
# target

In [24]:
target_names = ['negativo', 'positivo']

# Predisposizione del modello

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn


In [53]:
# https://www.kaggle.com/adamschroeder/countvectorizer-tfidfvectorizer-predict-comments

# *Max_df: When building the vocabulary, it ignores terms that have a document frequency 
# strictly higher than the given threshold. This could be used to exclude terms that are 
# too frequent and are unlikely to help predict the label. For example, by analyzing reviews 
# on the movie Lion King, the term 'Lion' might appear in 90% of the reviews (documents), in 
# which case, we could consider establishing Max_df=0.89

# **Min_df: ignores terms that have a document frequency (presence in % of documents) 
# strictly lower than the given threshold. For example, Min_df=0.66 requires that a 
# term appear in 66% of the docuemnts for it to be considered part of the vocabulary.
# Sometimes min_df is used to limit the vocabulary size, so it learns only those terms 
# that appear in at least 10%, 20%, etc. of the documents.

# **Max_features: Limit the amount of features (vocabulary) that the vectorizer will learn

# token_pattern=r'\w{2,}',  #vectorize 2-character words or more

tfidf_vectorizer = TfidfVectorizer(
    max_df=1.0, 
    min_df=1, 
    max_features=None,
    norm='l2',
    token_pattern=r'\w{2,}'
)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_dataset['description'])

In [54]:
#tfidf_vectorizer.get_feature_names()

### Trained model creation

In [55]:
mySVM = SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)

In [56]:
mySVM.fit(X_train_tfidf, train_dataset['classification'])

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=42, shuffle=True, tol=None, validation_fraction=0.1,
              verbose=0, warm_start=False)

### TFIDF transform of  TEST matrix

In [57]:
# X_test_tfidf = tfidf_vectorizer.fit_transform(test_dataset['description'])

# NEL DATASET DI TEST DEVO FARE SOLO IL TRANSFORM, NON IL FIT_TRANSFORM!!!
# (vedi sempre: https://www.kaggle.com/adamschroeder/countvectorizer-tfidfvectorizer-predict-comments, 
#               Capitolo: Clean, Train, Vectorize, Classify Toxic Comments (w/o parameter tuning)
#               Paragrafo: Vectorize)
X_test_tfidf = tfidf_vectorizer.transform(test_dataset['description']) 

### Prediction!

In [58]:
predicted_SVM = mySVM.predict(X_test_tfidf)

In [59]:
predicted_SVM.shape

(122,)

In [60]:
len(target)

122

In [61]:
np.mean(predicted_SVM == test_dataset['classification'])

0.7131147540983607

In [62]:
print(metrics.classification_report(target, predicted_SVM, target_names=target_names))

              precision    recall  f1-score   support

    negativo       0.74      0.60      0.66        57
    positivo       0.70      0.82      0.75        65

    accuracy                           0.71       122
   macro avg       0.72      0.71      0.71       122
weighted avg       0.72      0.71      0.71       122



### QUI ADESSO METTIAMO TUTTE LE RIGHE DI SOPRA INSIEME ALL'INTERNO DI UN CICLO NEL QUALE FACCIAMO VARIARE I VALORI DI TUTTI I PARAMETRI PER FARE UN HYPERPARAMETER TUNING

In [164]:
def hyperparameters_tuning(max_df, min_df, feat_perc, norm, file):    
    
    try:
        tmp_tfidf_vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df, max_features=None, norm=norm, token_pattern=r'\w{2,}')
        tmp_X_train_tfidf = tmp_tfidf_vectorizer.fit_transform(train_dataset['description'])
        num_features = len(tmp_tfidf_vectorizer.get_feature_names())
                        
        tfidf_vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df, max_features=int(num_features*feat_perc), norm=norm, token_pattern=r'\w{2,}')
        X_train_tfidf = tfidf_vectorizer.fit_transform(train_dataset['description'])
        X_test_tfidf = tfidf_vectorizer.transform(test_dataset['description']) 

        mySVM = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)
        mySVM.fit(X_train_tfidf, train_dataset['classification'])

        predicted_SVM = mySVM.predict(X_test_tfidf)
        summary_str = metrics.classification_report(target, predicted_SVM, target_names=target_names)
        summary_list = summary_str.split()

        negative_summary = {
            'precision': summary_list[5],
            'recall': summary_list[6],
            'f1-score': summary_list[7],
            'support': summary_list[8]
        }

        positive_summary = {
            'precision': summary_list[10],
            'recall': summary_list[11],
            'f1-score': summary_list[12],
            'support': summary_list[13]
        }

        accuracy_summary = {
            'f1-score': summary_list[15],
            'support': summary_list[16]    
        }

        summary = {
            'negative_summary': negative_summary,
            'positive_summary': positive_summary,
            'accuracy_summary': accuracy_summary
        }


#         print(
#             "{:.2f}".format(max_df),
#             "{:.2f}".format(min_df),
#             "{:.2f}".format(feat_perc),
#             num_features,
#             int(num_features*feat_perc),            
#             norm,
#             '\t',
#             summary['negative_summary']['precision'],
#             summary['negative_summary']['recall'],
#             summary['negative_summary']['f1-score'],
#             summary['negative_summary']['support'],
#             summary['positive_summary']['precision'],
#             summary['positive_summary']['recall'],
#             summary['positive_summary']['f1-score'],
#             summary['positive_summary']['support'],  
#             summary['accuracy_summary']['f1-score'], 
#             summary['accuracy_summary']['support']
#         )
        
        file.write(
            "{:.2f}".format(max_df)+','+
            "{:.2f}".format(min_df)+','+
            "{:.2f}".format(feat_perc)+','+
            str(num_features)+','+
            str(int(num_features*feat_perc))+','+            
            str(norm) +','+
            str(summary['negative_summary']['precision'])+','+
            str(summary['negative_summary']['recall'])+','+
            str(summary['negative_summary']['f1-score'])+','+
            str(summary['negative_summary']['support'])+','+
            str(summary['positive_summary']['precision'])+','+
            str(summary['positive_summary']['recall'])+','+
            str(summary['positive_summary']['f1-score'])+','+
            str(summary['positive_summary']['support'])+','+  
            str(summary['accuracy_summary']['f1-score'])+','+ 
            str(summary['accuracy_summary']['support'])+'\n'
        )        
        
    except Exception as e:
        dummy='dummy'
#         print(e)    

In [166]:
run=False

if run==True:
    file = open('hyperparameters_tuning.csv','w')

    # print('mxdf mndf ft% nfts maxft norm\t +prc +rec +f1s +s -prc -rec -f1s -s accf1 accsup')
    file.write('mxdf,mndf,ft%,nfts,maxft,norm,+prc,+rec,+f1s,+s,-prc,-rec,-f1s,-s,accf1,accsup\n')

    for max_df in sorted(numpy.arange(0, 1+0.05, 0.05), reverse=True):
        for min_df in sorted(numpy.arange(0, 1+0.05, 0.05), reverse=True):
            for norm in ['l1','l2',None]:
                for max_features in sorted(numpy.arange(0, 1+0.05, 0.05), reverse=True):
                    hyperparameters_tuning(max_df, min_df, max_features, norm, file)

    #                 break
    #             break
    #         break
    #     break

    file.close()

### Migliori parametri di TFIDF trovati con SGDClassifier:

In [ ]:
# Questi portano a un'accuracy di 0.74

# max_df = 1.0
# min_df = 0.0
# norm = None
# max_features = 0.65 #(65% del massimo numero delle feature, cioe' 1407 su un totale di 2166)

In [ ]:
# Questi portano a un'accuracy di 0.75 ma mi era sfuggito quindi ho fatto il resto 
# dell'analisi utilizzando i valori della cella di sopra ^_^*

# max_df = 0.1
# min_df = 0.0
# norm = l2
# max_features = 0.9 #(90% del massimo numero delle feature, cioe' 1935 su un totale di 2166)

### Adesso teniamo fissi i parametri di TFIDF  e proviamo diversi modelli di classificazione con diversi parametri

In [167]:
max_df = 1.0
min_df = 0.0
norm = None
max_features = 1407

tfidf_vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df, max_features=max_features, norm=norm, token_pattern=r'\w{2,}')
X_train_tfidf = tfidf_vectorizer.fit_transform(train_dataset['description'])
X_test_tfidf = tfidf_vectorizer.transform(test_dataset['description'])

In [168]:
def SGDClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file):    

    loss_values = ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
    penalty_values = ['l2', 'l1', 'elasticnet']
    alpha_values = [10 ** x for x in range(-6, 1)]
    max_iter = [10, 100, 500, 1000]
    learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive']
    
    for lo in loss_values:
        for p in penalty_values:
            for a in alpha_values:
                for m in max_iter:
                    for le in learning_rate:

                        try:

                            mySVM = SGDClassifier(loss=lo, penalty=p, alpha=a, random_state=42, max_iter=m, tol=None, learning_rate = le)
                            mySVM.fit(X_train_tfidf, train_dataset['classification'])

                            predicted_SVM = mySVM.predict(X_test_tfidf)
                            summary_str = metrics.classification_report(target, predicted_SVM, target_names=target_names)
                            summary_list = summary_str.split()

                            negative_summary = {
                                'precision': summary_list[5],
                                'recall': summary_list[6],
                                'f1-score': summary_list[7],
                                'support': summary_list[8]
                            }

                            positive_summary = {
                                'precision': summary_list[10],
                                'recall': summary_list[11],
                                'f1-score': summary_list[12],
                                'support': summary_list[13]
                            }

                            accuracy_summary = {
                                'f1-score': summary_list[15],
                                'support': summary_list[16]    
                            }

                            summary = {
                                'negative_summary': negative_summary,
                                'positive_summary': positive_summary,
                                'accuracy_summary': accuracy_summary
                            }


                            file.write(
                                lo+','+
                                p+','+
                                "{:.6f}".format(a)+','+
                                str(int(m))+','+            
                                le +','+
                                str(summary['negative_summary']['precision'])+','+
                                str(summary['negative_summary']['recall'])+','+
                                str(summary['negative_summary']['f1-score'])+','+
                                str(summary['negative_summary']['support'])+','+
                                str(summary['positive_summary']['precision'])+','+
                                str(summary['positive_summary']['recall'])+','+
                                str(summary['positive_summary']['f1-score'])+','+
                                str(summary['positive_summary']['support'])+','+  
                                str(summary['accuracy_summary']['f1-score'])+','+ 
                                str(summary['accuracy_summary']['support'])+'\n'
                            )        

                        except Exception as e:
                            dummy='dummy'
                    #         print(e)    

In [239]:
from sklearn.neighbors import KNeighborsClassifier

def KNeighborsClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file):    

    n_neighbors_values = [5, 25, 50]
    weights_values = ['uniform', 'distance']
    algorithm_values = ['auto', 'ball_tree', 'kd_tree', 'brute']
    leaf_size_values = [15, 30, 45, 60]
    
    for n in n_neighbors_values:
        for w in weights_values:
            for a in algorithm_values:
                for l in leaf_size_values:

                        try:

                            classifier = KNeighborsClassifier(n_neighbors=n, weights=w, algorithm=a, leaf_size=l)
                            classifier.fit(X_train_tfidf, train_dataset['classification'])

                            predicted = classifier.predict(X_test_tfidf)
                            summary_str = metrics.classification_report(target, predicted, target_names=target_names)
                            summary_list = summary_str.split()

                            negative_summary = {
                                'precision': summary_list[5],
                                'recall': summary_list[6],
                                'f1-score': summary_list[7],
                                'support': summary_list[8]
                            }

                            positive_summary = {
                                'precision': summary_list[10],
                                'recall': summary_list[11],
                                'f1-score': summary_list[12],
                                'support': summary_list[13]
                            }

                            accuracy_summary = {
                                'f1-score': summary_list[15],
                                'support': summary_list[16]    
                            }

                            summary = {
                                'negative_summary': negative_summary,
                                'positive_summary': positive_summary,
                                'accuracy_summary': accuracy_summary
                            }


                            file.write(
                                str(int(n))+','+ 
                                w+','+
                                a+','+
                                str(int(l))+','+            
                                str(summary['negative_summary']['precision'])+','+
                                str(summary['negative_summary']['recall'])+','+
                                str(summary['negative_summary']['f1-score'])+','+
                                str(summary['negative_summary']['support'])+','+
                                str(summary['positive_summary']['precision'])+','+
                                str(summary['positive_summary']['recall'])+','+
                                str(summary['positive_summary']['f1-score'])+','+
                                str(summary['positive_summary']['support'])+','+  
                                str(summary['accuracy_summary']['f1-score'])+','+ 
                                str(summary['accuracy_summary']['support'])+'\n'
                            )        
                            
                            if float(summary['accuracy_summary']['f1-score']) >= 0.74:
                                print(
                                    str(int(n))+','+ 
                                    w+','+
                                    a+','+
                                    str(int(l))+','+            
                                    str(summary['negative_summary']['precision'])+','+
                                    str(summary['negative_summary']['recall'])+','+
                                    str(summary['negative_summary']['f1-score'])+','+
                                    str(summary['negative_summary']['support'])+','+
                                    str(summary['positive_summary']['precision'])+','+
                                    str(summary['positive_summary']['recall'])+','+
                                    str(summary['positive_summary']['f1-score'])+','+
                                    str(summary['positive_summary']['support'])+','+  
                                    str(summary['accuracy_summary']['f1-score'])+','+ 
                                    str(summary['accuracy_summary']['support'])+'\n'                                    
                                )

                        except Exception as e:
                            dummy='dummy'
#                             print(e)    

In [240]:
from sklearn.tree import DecisionTreeClassifier

def DecisionTreeClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file):    

    criterion_values = ['gini', 'entropy']
    splitter_values = ['best', 'random']
    min_samples_split_values = [0.01, 0.02, 0.04, 0.08, 0.16]
    min_samples_leaf_values = [0.01, 0.02, 0.04, 0.08, 0.16]
    max_features_values = sorted(numpy.arange(0, 1+0.05, 0.05), reverse=True)
    
    for c in criterion_values:
        for s in splitter_values:
            for ms in min_samples_split_values:
                for ml in min_samples_leaf_values:
                    for mx in max_features_values:

                        try:

                            classifier = DecisionTreeClassifier(criterion=c, 
                                                              splitter=s, 
                                                              max_depth=None, 
                                                              min_samples_split=ms, 
                                                              min_samples_leaf=ml, 
                                                              min_weight_fraction_leaf=0.0, 
                                                              max_features=mx, 
                                                              random_state=None, 
                                                              max_leaf_nodes=None, 
                                                              min_impurity_decrease=0.0, 
                                                              min_impurity_split=None, 
                                                              class_weight=None, 
                                                              presort='deprecated', 
                                                              ccp_alpha=0.0)
                            classifier.fit(X_train_tfidf, train_dataset['classification'])

                            predicted = classifier.predict(X_test_tfidf)
                            summary_str = metrics.classification_report(target, predicted, target_names=target_names)
                            summary_list = summary_str.split()

                            negative_summary = {
                                'precision': summary_list[5],
                                'recall': summary_list[6],
                                'f1-score': summary_list[7],
                                'support': summary_list[8]
                            }

                            positive_summary = {
                                'precision': summary_list[10],
                                'recall': summary_list[11],
                                'f1-score': summary_list[12],
                                'support': summary_list[13]
                            }

                            accuracy_summary = {
                                'f1-score': summary_list[15],
                                'support': summary_list[16]    
                            }

                            summary = {
                                'negative_summary': negative_summary,
                                'positive_summary': positive_summary,
                                'accuracy_summary': accuracy_summary
                            }


                            file.write(
                                c+','+
                                s+','+
                                "{:.2f}".format(ms)+','+
                                "{:.2f}".format(ml)+','+
                                "{:.2f}".format(mx)+','+                                
                                str(summary['negative_summary']['precision'])+','+
                                str(summary['negative_summary']['recall'])+','+
                                str(summary['negative_summary']['f1-score'])+','+
                                str(summary['negative_summary']['support'])+','+
                                str(summary['positive_summary']['precision'])+','+
                                str(summary['positive_summary']['recall'])+','+
                                str(summary['positive_summary']['f1-score'])+','+
                                str(summary['positive_summary']['support'])+','+  
                                str(summary['accuracy_summary']['f1-score'])+','+ 
                                str(summary['accuracy_summary']['support'])+'\n'
                            )        

                            if float(summary['accuracy_summary']['f1-score']) >= 0.74:
                                print(
                                    c+','+
                                    s+','+
                                    "{:.2f}".format(ms)+','+
                                    "{:.2f}".format(ml)+','+
                                    "{:.2f}".format(mx)+','+                                
                                    str(summary['negative_summary']['precision'])+','+
                                    str(summary['negative_summary']['recall'])+','+
                                    str(summary['negative_summary']['f1-score'])+','+
                                    str(summary['negative_summary']['support'])+','+
                                    str(summary['positive_summary']['precision'])+','+
                                    str(summary['positive_summary']['recall'])+','+
                                    str(summary['positive_summary']['f1-score'])+','+
                                    str(summary['positive_summary']['support'])+','+  
                                    str(summary['accuracy_summary']['f1-score'])+','+ 
                                    str(summary['accuracy_summary']['support'])+'\n'                                
                                )
                            
                        except Exception as e:
                            dummy='dummy'
#                             print(e)    

In [283]:
from sklearn.ensemble import GradientBoostingClassifier

def GradientBoostingClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file):    

    loss_values = ['deviance', 'exponential']
    learning_rate_values = [0.1, 0.2, 0.4]
    n_estimators_values = [100, 500, 1000]
    min_samples_split_values = [0.01, 0.02, 0.04, 0.08, 0.16]
    min_samples_leaf_values = [0.01, 0.02, 0.04, 0.08, 0.16]    
    max_features_values = sorted(numpy.arange(0, 1+0.05, 0.05), reverse=True)
    warm_start_values = [True, False]
    
    for lo in loss_values:
        for le in learning_rate_values:
            for n in n_estimators_values:
                for ms in min_samples_split_values:
                    for ml in min_samples_leaf_values:
                        for mx in max_features_values:
                            for w in warm_start_values:

                                try:

                                    classifier = GradientBoostingClassifier(loss = lo,
                                                                            learning_rate = le,
                                                                            n_estimators = n,
                                                                            min_samples_split = ms,
                                                                            min_samples_leaf = ml,
                                                                            max_features = mx,
                                                                            warm_start = w                                                                    
                                    )
                                    classifier.fit(X_train_tfidf, train_dataset['classification'])

                                    predicted = classifier.predict(X_test_tfidf)
                                    summary_str = metrics.classification_report(target, predicted, target_names=target_names)
                                    summary_list = summary_str.split()

                                    negative_summary = {
                                        'precision': summary_list[5],
                                        'recall': summary_list[6],
                                        'f1-score': summary_list[7],
                                        'support': summary_list[8]
                                    }

                                    positive_summary = {
                                        'precision': summary_list[10],
                                        'recall': summary_list[11],
                                        'f1-score': summary_list[12],
                                        'support': summary_list[13]
                                    }

                                    accuracy_summary = {
                                        'f1-score': summary_list[15],
                                        'support': summary_list[16]    
                                    }

                                    summary = {
                                        'negative_summary': negative_summary,
                                        'positive_summary': positive_summary,
                                        'accuracy_summary': accuracy_summary
                                    }


                                    file.write(
                                        lo+','+
                                        "{:.2f}".format(le)+','+
                                        "{:.2f}".format(n)+','+
                                        "{:.2f}".format(ms)+','+  
                                        "{:.2f}".format(ml)+','+                                
                                        "{:.2f}".format(mx)+','+                                
                                        str(w)+','+                                                                        
                                        str(summary['negative_summary']['precision'])+','+
                                        str(summary['negative_summary']['recall'])+','+
                                        str(summary['negative_summary']['f1-score'])+','+
                                        str(summary['negative_summary']['support'])+','+
                                        str(summary['positive_summary']['precision'])+','+
                                        str(summary['positive_summary']['recall'])+','+
                                        str(summary['positive_summary']['f1-score'])+','+
                                        str(summary['positive_summary']['support'])+','+  
                                        str(summary['accuracy_summary']['f1-score'])+','+ 
                                        str(summary['accuracy_summary']['support'])+'\n'
                                    )        

                                    if float(summary['accuracy_summary']['f1-score']) >= 0.74:
                                        print(
                                            lo+','+
                                            "{:.2f}".format(le)+','+
                                            "{:.2f}".format(n)+','+
                                            "{:.2f}".format(ms)+','+  
                                            "{:.2f}".format(ml)+','+                                
                                            "{:.2f}".format(mx)+','+                                
                                            str(w)+','+                                                                        
                                            str(summary['negative_summary']['precision'])+','+
                                            str(summary['negative_summary']['recall'])+','+
                                            str(summary['negative_summary']['f1-score'])+','+
                                            str(summary['negative_summary']['support'])+','+
                                            str(summary['positive_summary']['precision'])+','+
                                            str(summary['positive_summary']['recall'])+','+
                                            str(summary['positive_summary']['f1-score'])+','+
                                            str(summary['positive_summary']['support'])+','+  
                                            str(summary['accuracy_summary']['f1-score'])+','+ 
                                            str(summary['accuracy_summary']['support'])+'\n'                               
                                        )

                                except Exception as e:
                                    dummy='dummy'
#                                     print(e)    

In [296]:
from sklearn.ensemble import RandomForestClassifier

def RandomForestClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file):    
   
    criterion_values = ['gini', 'entropy']
    min_samples_split_values = [0.01, 0.02, 0.04, 0.08, 0.16]
    min_samples_leaf_values = [0.01, 0.02, 0.04, 0.08, 0.16]
    max_features_values = sorted(numpy.arange(0, 1+0.05, 0.05), reverse=True)    
        
    for c in criterion_values:
            for ms in min_samples_split_values:
                for ml in min_samples_leaf_values:
                    for mx in max_features_values:

                                try:

                                    classifier = RandomForestClassifier(criterion = c,
                                                                        min_samples_split = ms,
                                                                        min_samples_leaf = ml,
                                                                        max_features = mx
                                                                       )
                                    classifier.fit(X_train_tfidf, train_dataset['classification'])

                                    predicted = classifier.predict(X_test_tfidf)
                                    summary_str = metrics.classification_report(target, predicted, target_names=target_names)
                                    summary_list = summary_str.split()

                                    negative_summary = {
                                        'precision': summary_list[5],
                                        'recall': summary_list[6],
                                        'f1-score': summary_list[7],
                                        'support': summary_list[8]
                                    }

                                    positive_summary = {
                                        'precision': summary_list[10],
                                        'recall': summary_list[11],
                                        'f1-score': summary_list[12],
                                        'support': summary_list[13]
                                    }

                                    accuracy_summary = {
                                        'f1-score': summary_list[15],
                                        'support': summary_list[16]    
                                    }

                                    summary = {
                                        'negative_summary': negative_summary,
                                        'positive_summary': positive_summary,
                                        'accuracy_summary': accuracy_summary
                                    }


                                    file.write(
                                        c+','+
                                        "{:.2f}".format(ms)+','+  
                                        "{:.2f}".format(ml)+','+                                
                                        "{:.2f}".format(mx)+','+                                
                                        str(summary['negative_summary']['precision'])+','+
                                        str(summary['negative_summary']['recall'])+','+
                                        str(summary['negative_summary']['f1-score'])+','+
                                        str(summary['negative_summary']['support'])+','+
                                        str(summary['positive_summary']['precision'])+','+
                                        str(summary['positive_summary']['recall'])+','+
                                        str(summary['positive_summary']['f1-score'])+','+
                                        str(summary['positive_summary']['support'])+','+  
                                        str(summary['accuracy_summary']['f1-score'])+','+ 
                                        str(summary['accuracy_summary']['support'])+'\n'
                                    )        

                                    if float(summary['accuracy_summary']['f1-score']) >= 0.74:
                                        print(
                                            c+','+
                                            "{:.2f}".format(ms)+','+  
                                            "{:.2f}".format(ml)+','+                                
                                            "{:.2f}".format(mx)+','+                                                                        
                                            str(summary['negative_summary']['precision'])+','+
                                            str(summary['negative_summary']['recall'])+','+
                                            str(summary['negative_summary']['f1-score'])+','+
                                            str(summary['negative_summary']['support'])+','+
                                            str(summary['positive_summary']['precision'])+','+
                                            str(summary['positive_summary']['recall'])+','+
                                            str(summary['positive_summary']['f1-score'])+','+
                                            str(summary['positive_summary']['support'])+','+  
                                            str(summary['accuracy_summary']['f1-score'])+','+ 
                                            str(summary['accuracy_summary']['support'])+'\n'                               
                                        )

                                except Exception as e:
                                    dummy='dummy'
#                                     print(e)    

In [310]:
from sklearn.neural_network import MLPClassifier #Multi-layer Perceptron classifier

def MLPClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file):    
   
#     criterion_values = ['gini', 'entropy']
#     min_samples_split_values = [0.01, 0.02, 0.04, 0.08, 0.16]
#     min_samples_leaf_values = [0.01, 0.02, 0.04, 0.08, 0.16]
#     max_features_values = sorted(numpy.arange(0, 1+0.05, 0.05), reverse=True) 

    hidden_layer_sizes_values = [100, 500, 1000]
    activation_values = ['identity', 'logistic', 'tanh', 'relu']
    solver_values = ['lbfgs', 'sgd', 'adam']
    learning_rate_values = ['constant', 'invscaling', 'adaptive']
    max_iter_values = [200, 400, 800]
        
    for h in hidden_layer_sizes_values:
        for a in activation_values:
            for s in solver_values:
                for l in learning_rate_values:
                    for m in max_iter_values:

                                try:

                                    classifier = MLPClassifier(
                                                                hidden_layer_sizes = h,
                                                                activation = a,
                                                                solver = s,
                                                                learning_rate = l,
                                                                max_iter = m
                                                               )
                                    classifier.fit(X_train_tfidf, train_dataset['classification'])

                                    predicted = classifier.predict(X_test_tfidf)
                                    summary_str = metrics.classification_report(target, predicted, target_names=target_names)
                                    summary_list = summary_str.split()

                                    negative_summary = {
                                        'precision': summary_list[5],
                                        'recall': summary_list[6],
                                        'f1-score': summary_list[7],
                                        'support': summary_list[8]
                                    }

                                    positive_summary = {
                                        'precision': summary_list[10],
                                        'recall': summary_list[11],
                                        'f1-score': summary_list[12],
                                        'support': summary_list[13]
                                    }

                                    accuracy_summary = {
                                        'f1-score': summary_list[15],
                                        'support': summary_list[16]    
                                    }

                                    summary = {
                                        'negative_summary': negative_summary,
                                        'positive_summary': positive_summary,
                                        'accuracy_summary': accuracy_summary
                                    }


                                    file.write(
                                        str(h)+','+
                                        a+','+
                                        s+','+
                                        l+','+
                                        str(m)+','+
                                        str(summary['negative_summary']['precision'])+','+
                                        str(summary['negative_summary']['recall'])+','+
                                        str(summary['negative_summary']['f1-score'])+','+
                                        str(summary['negative_summary']['support'])+','+
                                        str(summary['positive_summary']['precision'])+','+
                                        str(summary['positive_summary']['recall'])+','+
                                        str(summary['positive_summary']['f1-score'])+','+
                                        str(summary['positive_summary']['support'])+','+  
                                        str(summary['accuracy_summary']['f1-score'])+','+ 
                                        str(summary['accuracy_summary']['support'])+'\n'
                                    )        

                                    if float(summary['accuracy_summary']['f1-score']) >= 0.74:
                                        print(
                                            str(h)+','+
                                            a+','+
                                            s+','+
                                            l+','+
                                            str(m)+','+                                                                       
                                            str(summary['negative_summary']['precision'])+','+
                                            str(summary['negative_summary']['recall'])+','+
                                            str(summary['negative_summary']['f1-score'])+','+
                                            str(summary['negative_summary']['support'])+','+
                                            str(summary['positive_summary']['precision'])+','+
                                            str(summary['positive_summary']['recall'])+','+
                                            str(summary['positive_summary']['f1-score'])+','+
                                            str(summary['positive_summary']['support'])+','+  
                                            str(summary['accuracy_summary']['f1-score'])+','+ 
                                            str(summary['accuracy_summary']['support'])+'\n'                               
                                        )

                                except Exception as e:
                                    dummy='dummy'
#                                     print(e)    

In [276]:
# file_SGDClassifier_tuning = open('SGDClassifier_tuning.csv','w')
# file_SGDClassifier_tuning.write('lo,p,a,m,le,+prc,+rec,+f1s,+s,-prc,-rec,-f1s,-s,accf1,accsup\n')
# SGDClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file_SGDClassifier_tuning)
# file_SGDClassifier_tuning.close()

In [277]:
# file_KNeighborsClassifier_tuning = open('KNeighborsClassifier_tuning.csv','w')
# file_KNeighborsClassifier_tuning.write('n,w,a,l,+prc,+rec,+f1s,+s,-prc,-rec,-f1s,-s,accf1,accsup\n')
# KNeighborsClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file_KNeighborsClassifier_tuning)
# file_KNeighborsClassifier_tuning.close()

In [278]:
# file_DecisionTreeClassifier_tuning = open('DecisionTreeClassifier_tuning.csv','w')
# file_DecisionTreeClassifier_tuning.write('c,s,ms,ml,mx,+prc,+rec,+f1s,+s,-prc,-rec,-f1s,-s,accf1,accsup\n')
# DecisionTreeClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file_DecisionTreeClassifier_tuning)
# file_DecisionTreeClassifier_tuning.close()

In [284]:
# file_GradientBoostingClassifier_tuning = open('GradientBoostingClassifier_tuning.csv','w')
# file_GradientBoostingClassifier_tuning.write('lo,le,n,ms,ml,mx,w,+prc,+rec,+f1s,+s,-prc,-rec,-f1s,-s,accf1,accsup\n')
# GradientBoostingClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file_GradientBoostingClassifier_tuning)
# file_GradientBoostingClassifier_tuning.close()

deviance,0.10,100.00,0.01,0.01,1.00,False,0.73,0.70,0.71,57,0.75,0.77,0.76,65,0.74,122

deviance,0.10,100.00,0.01,0.01,0.85,False,0.75,0.68,0.72,57,0.74,0.80,0.77,65,0.75,122

deviance,0.10,100.00,0.01,0.01,0.80,True,0.75,0.68,0.72,57,0.74,0.80,0.77,65,0.75,122

deviance,0.10,100.00,0.01,0.01,0.55,False,0.76,0.65,0.70,57,0.73,0.82,0.77,65,0.74,122

deviance,0.10,100.00,0.01,0.01,0.50,False,0.74,0.68,0.71,57,0.74,0.78,0.76,65,0.74,122

deviance,0.10,100.00,0.01,0.01,0.20,False,0.75,0.67,0.70,57,0.73,0.80,0.76,65,0.74,122

deviance,0.10,100.00,0.01,0.01,0.05,True,0.75,0.67,0.70,57,0.73,0.80,0.76,65,0.74,122

deviance,0.10,100.00,0.01,0.01,0.05,False,0.76,0.67,0.71,57,0.74,0.82,0.77,65,0.75,122

deviance,0.10,100.00,0.02,0.01,1.00,True,0.75,0.67,0.70,57,0.73,0.80,0.76,65,0.74,122

deviance,0.10,100.00,0.02,0.01,0.90,False,0.74,0.68,0.71,57,0.74,0.78,0.76,65,0.74,122

deviance,0.10,100.00,0.02,0.01,0.85,False,0.73,0.70,0.71,57,0.75,0.77,0.76,65,0.74,122

deviance,0.10,100.00,0.02,0.01,0.75

deviance,0.10,500.00,0.16,0.01,0.15,True,0.73,0.70,0.71,57,0.75,0.77,0.76,65,0.74,122

deviance,0.10,500.00,0.16,0.01,0.10,True,0.74,0.68,0.71,57,0.74,0.78,0.76,65,0.74,122

deviance,0.10,500.00,0.16,0.01,0.05,True,0.75,0.68,0.72,57,0.74,0.80,0.77,65,0.75,122

deviance,0.10,500.00,0.16,0.01,0.05,False,0.75,0.67,0.70,57,0.73,0.80,0.76,65,0.74,122

deviance,0.10,1000.00,0.01,0.01,0.05,False,0.72,0.74,0.73,57,0.77,0.75,0.76,65,0.75,122

deviance,0.10,1000.00,0.02,0.01,0.60,False,0.71,0.74,0.72,57,0.76,0.74,0.75,65,0.74,122

deviance,0.10,1000.00,0.16,0.01,0.30,True,0.71,0.74,0.72,57,0.76,0.74,0.75,65,0.74,122

deviance,0.20,100.00,0.01,0.01,1.00,True,0.75,0.74,0.74,57,0.77,0.78,0.78,65,0.76,122

deviance,0.20,100.00,0.01,0.01,1.00,False,0.75,0.74,0.74,57,0.77,0.78,0.78,65,0.76,122

deviance,0.20,100.00,0.01,0.01,0.90,False,0.78,0.68,0.73,57,0.75,0.83,0.79,65,0.76,122

deviance,0.20,100.00,0.01,0.01,0.85,True,0.71,0.74,0.72,57,0.76,0.74,0.75,65,0.74,122

deviance,0.20,100.00,0.01,0.01,0.75

deviance,0.20,100.00,0.16,0.01,0.25,True,0.75,0.68,0.72,57,0.74,0.80,0.77,65,0.75,122

deviance,0.20,100.00,0.16,0.01,0.25,False,0.78,0.67,0.72,57,0.74,0.83,0.78,65,0.75,122

deviance,0.20,100.00,0.16,0.01,0.20,True,0.75,0.72,0.73,57,0.76,0.78,0.77,65,0.75,122

deviance,0.20,100.00,0.16,0.01,0.20,False,0.74,0.70,0.72,57,0.75,0.78,0.77,65,0.75,122

deviance,0.20,100.00,0.16,0.01,0.15,True,0.78,0.67,0.72,57,0.74,0.83,0.78,65,0.75,122

deviance,0.20,100.00,0.16,0.01,0.10,True,0.75,0.70,0.73,57,0.75,0.80,0.78,65,0.75,122

deviance,0.20,100.00,0.16,0.01,0.10,False,0.79,0.65,0.71,57,0.73,0.85,0.79,65,0.75,122

deviance,0.20,100.00,0.16,0.01,0.05,True,0.74,0.70,0.72,57,0.75,0.78,0.77,65,0.75,122

deviance,0.20,100.00,0.16,0.01,0.05,False,0.74,0.74,0.74,57,0.77,0.77,0.77,65,0.75,122

deviance,0.20,500.00,0.16,0.01,0.35,True,0.71,0.74,0.72,57,0.76,0.74,0.75,65,0.74,122

deviance,0.40,100.00,0.01,0.01,0.90,False,0.70,0.75,0.73,57,0.77,0.72,0.75,65,0.74,122

deviance,0.40,100.00,0.01,0.01,0.75,Fa

exponential,0.10,500.00,0.16,0.01,0.45,True,0.73,0.70,0.71,57,0.75,0.77,0.76,65,0.74,122

exponential,0.10,500.00,0.16,0.01,0.40,True,0.73,0.70,0.71,57,0.75,0.77,0.76,65,0.74,122

exponential,0.10,500.00,0.16,0.01,0.25,True,0.74,0.68,0.71,57,0.74,0.78,0.76,65,0.74,122

exponential,0.10,500.00,0.16,0.01,0.20,True,0.74,0.68,0.71,57,0.74,0.78,0.76,65,0.74,122

exponential,0.10,500.00,0.16,0.01,0.10,True,0.77,0.70,0.73,57,0.76,0.82,0.79,65,0.76,122

exponential,0.10,500.00,0.16,0.01,0.10,False,0.75,0.68,0.72,57,0.74,0.80,0.77,65,0.75,122

exponential,0.10,500.00,0.16,0.01,0.05,False,0.74,0.68,0.71,57,0.74,0.78,0.76,65,0.74,122

exponential,0.10,1000.00,0.01,0.01,0.60,True,0.70,0.75,0.73,57,0.77,0.72,0.75,65,0.74,122

exponential,0.10,1000.00,0.02,0.01,0.05,False,0.73,0.70,0.71,57,0.75,0.77,0.76,65,0.74,122

exponential,0.20,100.00,0.01,0.01,1.00,False,0.75,0.74,0.74,57,0.77,0.78,0.78,65,0.76,122

exponential,0.20,100.00,0.01,0.01,0.70,False,0.76,0.68,0.72,57,0.75,0.82,0.78,65,0.75,122

exp

exponential,0.40,100.00,0.02,0.01,0.70,False,0.73,0.70,0.71,57,0.75,0.77,0.76,65,0.74,122

exponential,0.40,100.00,0.02,0.01,0.65,True,0.73,0.75,0.74,57,0.78,0.75,0.77,65,0.75,122

exponential,0.40,100.00,0.02,0.01,0.45,False,0.72,0.72,0.72,57,0.75,0.75,0.75,65,0.74,122

exponential,0.40,100.00,0.02,0.01,0.15,False,0.75,0.70,0.73,57,0.75,0.80,0.78,65,0.75,122

exponential,0.40,100.00,0.02,0.01,0.10,True,0.74,0.68,0.71,57,0.74,0.78,0.76,65,0.74,122

exponential,0.40,100.00,0.04,0.01,0.65,False,0.73,0.70,0.71,57,0.75,0.77,0.76,65,0.74,122

exponential,0.40,100.00,0.04,0.01,0.60,True,0.73,0.70,0.71,57,0.75,0.77,0.76,65,0.74,122

exponential,0.40,100.00,0.04,0.01,0.50,False,0.73,0.70,0.71,57,0.75,0.77,0.76,65,0.74,122

exponential,0.40,100.00,0.04,0.01,0.40,True,0.74,0.68,0.71,57,0.74,0.78,0.76,65,0.74,122

exponential,0.40,100.00,0.04,0.01,0.10,True,0.74,0.70,0.72,57,0.75,0.78,0.77,65,0.75,122

exponential,0.40,100.00,0.04,0.01,0.05,True,0.74,0.70,0.72,57,0.75,0.78,0.77,65,0.75,122

expon

In [297]:
# file_RandomForestClassifier_tuning = open('RandomForestClassifier_tuning.csv','w')
# file_RandomForestClassifier_tuning.write('c,s,ms,ml,mx,+prc,+rec,+f1s,+s,-prc,-rec,-f1s,-s,accf1,accsup\n')
# RandomForestClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file_RandomForestClassifier_tuning)
# file_RandomForestClassifier_tuning.close()

entropy,0.16,0.01,0.15,0.76,0.65,0.70,57,0.73,0.82,0.77,65,0.74,122



In [311]:
# file_MLPClassifier_tuning = open('MLPClassifier_tuning.csv','w')
# file_MLPClassifier_tuning.write('h,a,s,l,m,+prc,+rec,+f1s,+s,-prc,-rec,-f1s,-s,accf1,accsup\n')
# MLPClassifier_tuning(tfidf_vectorizer, X_train_tfidf, X_test_tfidf, file_MLPClassifier_tuning)
# file_MLPClassifier_tuning.close()

100,logistic,sgd,constant,800,0.78,0.61,0.69,57,0.71,0.85,0.77,65,0.74,122

100,logistic,sgd,adaptive,800,0.80,0.63,0.71,57,0.73,0.86,0.79,65,0.75,122

100,tanh,sgd,constant,200,0.77,0.63,0.69,57,0.72,0.83,0.77,65,0.74,122

100,relu,lbfgs,invscaling,800,0.77,0.65,0.70,57,0.73,0.83,0.78,65,0.75,122

500,logistic,sgd,constant,800,0.79,0.65,0.71,57,0.73,0.85,0.79,65,0.75,122

500,logistic,sgd,adaptive,800,0.77,0.63,0.69,57,0.72,0.83,0.77,65,0.74,122

500,relu,lbfgs,constant,800,0.78,0.61,0.69,57,0.71,0.85,0.77,65,0.74,122

500,relu,lbfgs,adaptive,400,0.78,0.61,0.69,57,0.71,0.85,0.77,65,0.74,122

500,relu,sgd,constant,200,0.80,0.58,0.67,57,0.70,0.88,0.78,65,0.74,122

500,relu,sgd,constant,400,0.81,0.61,0.70,57,0.72,0.88,0.79,65,0.75,122

500,relu,sgd,adaptive,800,0.79,0.60,0.68,57,0.71,0.86,0.78,65,0.74,122

1000,identity,adam,adaptive,800,0.79,0.60,0.68,57,0.71,0.86,0.78,65,0.74,122

1000,logistic,sgd,constant,400,0.82,0.63,0.71,57,0.73,0.88,0.80,65,0.76,122

1000,logistic,sgd,constant,80

In [ ]:
### Stima dei parametri medi mediante utilizzo di Grid Search con Cross-Validation 

# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report
# from sklearn.svm import SVC

# # Set the parameters by cross-validation
# tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 100, 1000]},
#                     {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

# X_train = X_train_tfidf # nota che questa e' la matrice tfidf e contiene anche feature numeriche
# y_train = train_dataset['classification']

# X_test = X_test_tfidf # nota che questa e' la matrice tfidf RIORGANIZZATA e contiene anche feature numeriche
# y_test = test_dataset['classification']

# scores = ['precision', 'recall']

# for score in scores:
#     print("# Tuning hyper-parameters for %s" % score)
#     print()

#     clf = GridSearchCV(
#         SVC(), tuned_parameters, scoring='%s_macro' % score
#     )
#     clf.fit(X_train, y_train)

#     print("Best parameters set found on development set:")
#     print()
#     print(clf.best_params_)
#     print()
#     print("Grid scores on development set:")
#     print()
#     means = clf.cv_results_['mean_test_score']
#     stds = clf.cv_results_['std_test_score']
#     for mean, std, params in zip(means, stds, clf.cv_results_['params']):
#         print("%0.3f (+/-%0.03f) for %r"
#               % (mean, std * 2, params))
#     print()

#     print("Detailed classification report:")
#     print()
#     print("The model is trained on the full development set.")
#     print("The scores are computed on the full evaluation set.")
#     print()
#     y_true, y_pred = y_test, clf.predict(X_test)
#     print(classification_report(y_true, y_pred))
#     print()



In [427]:
# NOTA:   I VALORI 0.92,0.63,0.75,57,0.75,0.95,0.84,65,0.80,122 LI HO OTTENUTI 
#         DURANTE IL TUNING DEGLI IPERPARAMETRI CON I SEGUENTI VALORI DEI PARAMETRI
#         DEL CLASSIFICATORE DecisionTreeClassifier:
#             criterion='gini', 
#             splitter='random', 
#             min_samples_split=0.16, 
#             min_samples_leaf=0.04, 
#             max_features=0.9
#         HO SCOPERTO PERO' CHE SI POSSONO RIOTTENERE ANCHE CON I VALORI:
#             criterion='gini', 
#             splitter='best', 
#             min_samples_split=0.02, 
#             min_samples_leaf=0.04, 
#             max_features=0.55,       
#         A PATTO DI IMPOSTARE: 
#             random_state=581
#         DI CONSEGUENZA USERO' QUESTI PARAMETRI PER IL CLASSIFICATORE
    

from sklearn.tree import DecisionTreeClassifier

for i in range(1000):
    classifier = DecisionTreeClassifier(
                                        criterion='gini', 
                                        splitter='best', 
                                        min_samples_split=0.02, 
                                        min_samples_leaf=0.04, 
                                        min_weight_fraction_leaf=0.0, 
                                        max_features=0.55,
                                        random_state=i, 
                                        max_leaf_nodes=None, 
                                        min_impurity_decrease=0.0, 
                                        min_impurity_split=None, 
                                        class_weight=None, 
                                        presort='deprecated', 
                                        ccp_alpha=0.0
    )
    classifier.fit(X_train_tfidf, train_dataset['classification'])

    predicted = classifier.predict(X_test_tfidf)
    summary_str = metrics.classification_report(target, predicted, target_names=target_names)

    summary_list = summary_str.split()

    negative_summary = {
        'precision': summary_list[5],
        'recall': summary_list[6],
        'f1-score': summary_list[7],
        'support': summary_list[8]
    }

    positive_summary = {
        'precision': summary_list[10],
        'recall': summary_list[11],
        'f1-score': summary_list[12],
        'support': summary_list[13]
    }

    accuracy_summary = {
        'f1-score': summary_list[15],
        'support': summary_list[16]    
    }

    summary = {
        'negative_summary': negative_summary,
        'positive_summary': positive_summary,
        'accuracy_summary': accuracy_summary
    }
    
    if float(summary['accuracy_summary']['f1-score']) >= 0.80:
        print(
            str(i)+','+                        
            str(summary['negative_summary']['precision'])+','+
            str(summary['negative_summary']['recall'])+','+
            str(summary['negative_summary']['f1-score'])+','+
            str(summary['negative_summary']['support'])+','+
            str(summary['positive_summary']['precision'])+','+
            str(summary['positive_summary']['recall'])+','+
            str(summary['positive_summary']['f1-score'])+','+
            str(summary['positive_summary']['support'])+','+  
            str(summary['accuracy_summary']['f1-score'])+','+ 
            str(summary['accuracy_summary']['support'])+'\n'                                
        )

294,0.90,0.63,0.74,57,0.74,0.94,0.83,65,0.80,122

321,0.88,0.67,0.76,57,0.76,0.92,0.83,65,0.80,122

349,0.88,0.67,0.76,57,0.76,0.92,0.83,65,0.80,122

581,0.92,0.63,0.75,57,0.75,0.95,0.84,65,0.80,122



In [441]:
classifier = DecisionTreeClassifier(
                                        criterion='gini', 
                                        splitter='best', 
                                        min_samples_split=0.02, 
                                        min_samples_leaf=0.04, 
                                        min_weight_fraction_leaf=0.0, 
                                        max_features=0.55,
                                        random_state=581, 
                                        max_leaf_nodes=None, 
                                        min_impurity_decrease=0.0, 
                                        min_impurity_split=None, 
                                        class_weight=None, 
                                        presort='deprecated', 
                                        ccp_alpha=0.0
)
classifier.fit(X_train_tfidf, train_dataset['classification'])

predicted = classifier.predict(X_test_tfidf)
summary_str = metrics.classification_report(target, predicted, target_names=target_names)
print(summary_str)

              precision    recall  f1-score   support

    negativo       0.92      0.63      0.75        57
    positivo       0.75      0.95      0.84        65

    accuracy                           0.80       122
   macro avg       0.84      0.79      0.79       122
weighted avg       0.83      0.80      0.80       122



In [443]:
from sklearn.externals import joblib

# joblib.dump(clf.best_estimator_, 'model_reduced.pkl')
# joblib.dump(clf.best_estimator_, 'model_reduced_IT.pkl')
# joblib.dump(clf.best_estimator_, 'model_reduced_EN.pkl')

joblib.dump(classifier, 'model_reduced_DecisionTree_EN.pkl')

['model_reduced_DecisionTree_EN.pkl']

In [445]:
# file = open('model_features_reduced.dat','w')
# file = open('model_features_IT_reduced.dat','w')
# file = open('model_features_EN_reduced.dat','w')

file = open('model_features_DecisionTree_EN_reduced.dat','w')

cnt=0
for i in tfidf_vectorizer.get_feature_names():
    file.write(i+'\n')
    cnt+=1
    
print("Wrote",cnt,"features")
    
file.close()

Wrote 1407 features
